# Introduction to Kubernetes

**Learning Objectives**
 * Create GKE cluster from command line
 * Deploy an application to your cluster
 * Cleanup, delete the cluster 

## Overview
Kubernetes is an open source project (available on [kubernetes.io](kubernetes.io)) which can run on many different environments, from laptops to high-availability multi-node clusters; from public clouds to on-premise deployments; from virtual machines to bare metal.

The goal of this lab is to provide a short introduction to Kubernetes (k8s) and some basic functionality.

## Create a GKE cluster

A cluster consists of at least one cluster master machine and multiple worker machines called nodes. Nodes are Compute Engine virtual machine (VM) instances that run the Kubernetes processes necessary to make them part of the cluster.

**Note**: Cluster names must start with a letter and end with an alphanumeric, and cannot be longer than 40 characters.

We'll call our cluster `asl-cluster`.

In [5]:
import os

CLUSTER_NAME = "asl-cluster"
ZONE = "us-central1-a"

os.environ["CLUSTER_NAME"] = CLUSTER_NAME
os.environ["ZONE"] = ZONE

We'll set our default compute zone to `us-central1-a` and use `gcloud container clusters create ...` to create the GKE cluster. Let's first look at all the clusters we currently have. 

In [6]:
!gcloud container clusters list

**Exercise**

Use `gcloud container clusters create` to create a new cluster using the `CLUSTER_NAME` we set above. This takes a few minutes...

In [12]:
!gcloud container clusters create $CLUSTER_NAME --zone us-central1-a

Default change: VPC-native is the default mode during cluster creation for versions greater than 1.21.0-gke.1500. To create advanced routes based clusters, please pass the `--no-enable-ip-alias` flag
Note: Your Pod address range (`--cluster-ipv4-cidr`) can accommodate at most 1008 node(s).
Creating cluster asl-cluster in us-central1-a... Cluster is being configured...
⠛                                                                              
Creating cluster asl-cluster in us-central1-a... Cluster is being health-checke
d (master is healthy)...done.                                                  
Created [https://container.googleapis.com/v1/projects/qwiklabs-gcp-01-9a9d18213c32/zones/us-central1-a/clusters/asl-cluster].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/us-central1-a/asl-cluster?project=qwiklabs-gcp-01-9a9d18213c32
kubeconfig entry generated for asl-cluster.
NAME         LOCATION       MASTER_VERSION   MA

Now when we list our clusters again, we should see the cluster we created. 

In [14]:
!gcloud container clusters list

NAME         LOCATION       MASTER_VERSION   MASTER_IP     MACHINE_TYPE  NODE_VERSION     NUM_NODES  STATUS
asl-cluster  us-central1-a  1.21.9-gke.1002  34.121.57.47  e2-medium     1.21.9-gke.1002  3          RUNNING


## Get authentication credentials and deploy and application

After creating your cluster, you need authentication credentials to interact with it. Use `get-credentials` to authenticate the cluster.

**Exercise**

Use `gcloud container clusters get-credentials` to authenticate the cluster you created.

In [17]:
!gcloud container clusters get-credentials $CLUSTER_NAME --zone $ZONE

Fetching cluster endpoint and auth data.
kubeconfig entry generated for asl-cluster.


You can now deploy a containerized application to the cluster. For this lab, you'll run `hello-app` in your cluster.

GKE uses Kubernetes objects to create and manage your cluster's resources. Kubernetes provides the [Deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/) object for deploying stateless applications like web servers. [Service](https://kubernetes.io/docs/concepts/services-networking/service/) objects define rules and load balancing for accessing your application from the internet.

**Exercise**

Use the `kubectl create` command to create a new Deployment `hello-server` from the `hello-app` container image. The `--image` flag to specify a container image to deploy. The `kubectl create` command pulls the example image from a Container Registry bucket. Here, use [gcr.io/google-samples/hello-app:1.0](gcr.io/google-samples/hello-app:1.0) to indicate the specific image version to pull. If a version is not specified, the latest version is used.

In [18]:
!kubectl create deployment hello-server --image gcr.io/google-samples/hello-app:1.0

deployment.apps/hello-server created


This Kubernetes command creates a Deployment object that represents `hello-server`. To create a Kubernetes Service, which is a Kubernetes resource that lets you expose your application to external traffic, run the `kubectl expose` command. 

**Exercise**

Use the `kubectl expose` to expose the application. In this command, 
  * `--port` specifies the port that the container exposes.
  * `type="LoadBalancer"` creates a Compute Engine load balancer for your container.

In [21]:
!kubectl expose deployment hello-server --type=LoadBalancer --name=my-service --port=8000

service/my-service exposed


Use the `kubectl get service` command to inspect the `hello-server` Service.

**Note**: It might take a minute for an external IP address to be generated. Run the previous command again if the `EXTERNAL-IP` column for `hello-server` status is pending.

In [25]:
!kubectl get service

NAME         TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)          AGE
kubernetes   ClusterIP      10.88.0.1      <none>          443/TCP          5m20s
my-service   LoadBalancer   10.88.15.171   34.123.81.148   8000:31844/TCP   75s


You can now view the application from your web browser, open a new tab and enter the following address, replacing `EXTERNAL IP` with the EXTERNAL-IP for `hello-server`:

```bash
http://[EXTERNAL_IP]:8080
```

You should see a simple page which displays

```bash
Hello, world!
Version: 1.0.0
Hostname: hello-server-5bfd595c65-7jqkn
```

## Cleanup

Delete the cluster using `gcloud` to free up those resources. Use the `--quiet` flag if you are executing this in a notebook. Deleting the cluster can take a few minutes. 

**Exercise**

Delete the cluster. Use the `--quiet` flag since we're executing in a notebook.

In [26]:
!gcloud container clusters delete $CLUSTER_NAME --zone $ZONE --quiet

Deleting cluster asl-cluster...done.                                           
Deleted [https://container.googleapis.com/v1/projects/qwiklabs-gcp-01-9a9d18213c32/zones/us-central1-a/clusters/asl-cluster].


Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.